In [1]:
import csv
import pandas as pd
import numpy
import math
from heapq import heappush, heappop
import collections


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filename = "100 nodes-1.csv"

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_project


/content/drive/MyDrive/Colab Notebooks/AI_project


In [5]:
with open(filename) as csvfile:
    csvreader = csv.DictReader(csvfile,delimiter=',')

df = pd.read_csv(filename)
df[:5]

,Unnamed: 0,x,y,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,10,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,20,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,30,0,0,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,40,0,0,0,0,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:


class Node:
    def __init__(self, state, parent=None, g_cost=0, h_cost=0 , f_cost=0):
        self.state   = state
        if state >= 0 and state < len(df):
            self.x = df.iloc[state,1]
            self.y = df.iloc[state,2]
        self.parent = parent
        self.g_cost = g_cost
        self.h_cost = h_cost
        self.f_cost = f_cost
        

    def __lt__(self, other):
        return self.f_cost < other.f_cost

    def __repr__(self):
        return str(self.state)

In [10]:
def H_cost(current_node, goal_node):
    if current_node.state >= len(df) or goal_node.state >= len(df):
        return 0
    # Calculate the Euclidean distance between the current node and the goal node
    h_cost = (abs(current_node.x - goal_node.x) + abs(current_node.y - goal_node.y))  
    return h_cost

def F_cost(current_node, goal_node, weight):
    return (current_node.g_cost * weight) + (H_cost(current_node, goal_node) * (1 - weight))



In [66]:
def A_star(start, goal, weight):
    open_set = []
    closed_set = []
    start_node = Node(start)
    goal_node = Node(goal)
    heappush(open_set, start_node)
    min_f_cost = float("inf")
    max_iterations = 1000
    iteration = 0

    while open_set and iteration < max_iterations:
          #print("Open set:", [node.state for node in open_set])
          current_node = min(open_set, key=lambda x: x.f_cost)  # get the node with the lowest f_cost
          #print("Current node:", current_node.state)
          if current_node.state == goal:
              return get_path(current_node), current_node.g_cost

          open_set.remove(current_node)  # remove the node from the open_set
          closed_set.append(current_node.state)
          #print("Closed set:", closed_set)

          for neighbor in get_neighbors(current_node.state, current_node, closed_set):
              if neighbor in closed_set:
                  continue

              neighbor.g_cost = current_node.g_cost + df.iloc[current_node.state,neighbor.state + 3]
              neighbor.h_cost = H_cost(neighbor, goal_node)
              neighbor.f_cost = F_cost(neighbor, goal_node, weight)
              neighbor_node = neighbor

              for node in open_set:
                  if node.state == neighbor.state and node.f_cost > neighbor.f_cost:
                      node.f_cost = neighbor.f_cost
                      node.g_cost = neighbor.g_cost
                      node.h_cost = neighbor.h_cost
                      node.parent = neighbor.parent
                      break
              else:
                  heappush(open_set, neighbor)

              iteration += 1
              if iteration == max_iterations:
                 return "There is no path", 0


    # return "There is no path", 0

def get_neighbors(state, parent, closed_set):
    neighbors = [state+1, state-1, state+10, state-10]
    new_neighbors = []
    for i in neighbors:
        if i >= 0 and i < 100 and df.iloc[state,i + 3] != 0: #and i not in closed_set:
            new_neighbors.append(Node(i, parent=parent))
    return new_neighbors





In [50]:
def get_path(node):
    path = []
    current = node
    while current is not None:
        path.append(current.state)
        current = current.parent

    return path[::-1]

In [51]:
start = 0
goal = 19
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[0, 10, 11, 12, 13, 14, 24, 25, 15, 16, 17, 18, 19] 95 13


In [52]:
start = 0
goal = 19
weight = 0

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[0, 1, 2, 12, 13, 14, 24, 25, 15, 16, 17, 18, 19] 96 13


In [55]:
start = 11
goal = 97
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[11, 21, 31, 32, 33, 34, 44, 45, 46, 56, 66, 76, 77, 87, 97] 105 15


In [56]:
start = 11
goal = 97
weight = 0.25

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[11, 21, 31, 32, 33, 34, 44, 45, 46, 56, 66, 76, 77, 87, 97] 105 15


In [67]:
start = 40
goal = 49
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[40, 41, 51, 52, 42, 43, 44, 45, 46, 47, 37, 38, 28, 29, 39, 49] 109 16


In [68]:
start = 49
goal = 40
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[49, 59, 58, 48, 38, 37, 36, 35, 34, 33, 32, 31, 30, 40] 101 14


In [69]:
start = 0
goal = 99
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

There is no path 0 16


In [70]:
start = 99
goal = 0
weight = 0.5

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[99, 98, 97, 87, 77, 76, 75, 65, 64, 54, 53, 52, 51, 41, 40, 30, 20, 10, 0] 138 19


In [79]:
start = 99
goal = 0
weight = 0.25

path, cost = A_star(start, goal, weight)

print(path,cost,len(path))

[99, 98, 97, 87, 77, 76, 75, 65, 64, 54, 53, 52, 51, 41, 40, 30, 20, 10, 0] 138 19
